In [101]:
import pandas as pd
import numpy as np
%matplotlib inline

In [102]:
data = pd.read_csv('data/data_update-2017-10.csv')

In [103]:
import unicodedata

def elimina_tildes(s):
    return ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))

def descripcion_token(x):
    x = elimina_tildes(x)
    words = ['parrilla','suite','cochera', 'placard', 'pileta', 'terraza',\
             'vestidor', 'patio', 'hall', 'servicio', 'muebles', 'balcon', 'jard','aire acondicionado', \
             'quincho', 'fondo', 'piscina', 'garage', 'parque', 'estrenar', \
             'galer', 'dependencia', 'buena','calefaccion','radiante', 'seguridad', \
             'chalet','gimnasio','vista','calidad','centro','amplio','luminoso','solar','entrega']
    result = ''
    for word in words:
        if word in x:
            result += '|' + word
    return result
def place_name_from_parent(x): ## Segun vi el 3 item del place es generalmente el lugar
    list = x.split('|')
    if(len(list) < 5):
        return np.nan
    else:
        return list[3]
    
def rescatar_barrios_no_oficiales(x):
    if x == 'Once':
        return ("Balvanera")
    elif x == 'Bajo Belgrano':
        return ("Belgrano")
    elif x == 'Alto Palermo':
        return ("Palermo")
    elif x == 'Tribunales':
        return ("San Nicolás")
    elif x == 'Las Cañitas':
        return("Palermo")
    elif x == 'Capital Federal':
        return(np.nan)
    else:
        return x
    
def recuperar_cant_amb(x):
    "Recupera el dato amb en x (puede ser la descripcion o el titulo)"
    try:
        pos = 0
        x = x.lower()
        if "ambientes" in x:
            pos = x.rfind("ambientes")
        elif "amb" in x:
            pos = x.rfind("amb")
        if pos != 0:
            try:
                return (np.float64(x[pos -2]))
            except ValueError:
                if ("dos amb" in x) or ("dos ambientes" in x):
                    return 2.0
                elif ("tres amb" in x) or ("tres ambientes" in x):
                    return 3.0
                elif ("cuatro amb" in x) or ("cuatro ambientes" in x):
                    return 4.0
                elif ("cinco amb" in x) or ("cinco ambientes" in x):
                    return 5.0
                elif ("seis amb" in x) or ("seis ambientes" in x):
                    return 6.0
                return np.nan
    except AttributeError:
        return None
import re
def precio_from_desc(x):
    try:
        x = x.lower().replace(".","")
        str_price = re.findall("u[s\$][sd] *(\d+)", x)
        if (len(str_price) == 0):
            str_price = re.findall("(\d+) *u[s\$][sd]", x)
        if (len(str_price) > 0):            
            p = float(str_price[0])
            if (p > 2700000.0 or p <= 20000.0):
                    return np.nan
            return p
        return np.nan
    except:
        return np.nan


In [104]:
precio_from_desc("usd 400.000 asdasd")

400000.0

In [105]:
#data['precio_from_desc'] = data.description.apply(precio_from_desc)
data.loc[:,'place_name'] = data.place_with_parent_names.apply(place_name_from_parent) ## esto quita muchos problemas 
                                                                                    ## de place name como nombres de country
                                                                                    ## o palermo soho
data.loc[data.description.notnull(),'description'] = data.loc[data.description.notnull(),'description'].apply(descripcion_token)

In [106]:
data.loc[:,'place_name'] = data.place_name.apply(rescatar_barrios_no_oficiales)
data['created_on'] = pd.to_datetime(data['created_on']) ## conversion de tipo
data['month'] = data.created_on.apply(lambda x: x.month)
data['year'] = data.created_on.apply(lambda x: x.year)
data['day'] = data.created_on.apply(lambda x: x.day)

In [107]:
data = data[~data.property_type.str.contains('store')]

In [108]:
data.property_type.value_counts()

apartment    176967
house        111032
PH            19100
Name: property_type, dtype: int64

In [109]:
data.dropna(axis=0,subset=['price_aprox_usd'],inplace=True) ## drop de los datos sin precio
data = data.loc[data['price_aprox_usd'] > 0,:]

In [110]:
data.loc[data.rooms.isnull(),'rooms'] = data.description.apply(recuperar_cant_amb)
data.loc[data.rooms.isnull(),'rooms'] = data.title.apply(recuperar_cant_amb)
data.loc[data.surface_total_in_m2.isnull(),'surface_total_in_m2'] = data.surface_covered_in_m2

In [111]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276246 entries, 0 to 317037
Data columns (total 26 columns):
id                            276246 non-null object
created_on                    276246 non-null datetime64[ns]
property_type                 276246 non-null object
place_name                    270199 non-null object
place_with_parent_names       276246 non-null object
state_name                    276246 non-null object
geonames_id                   217095 non-null float64
lat-lon                       196422 non-null object
lat                           196422 non-null float64
lon                           196422 non-null float64
price                         276246 non-null float64
currency                      276246 non-null object
price_aprox_local_currency    276246 non-null float64
price_aprox_usd               276246 non-null float64
surface_total_in_m2           258993 non-null float64
surface_covered_in_m2         250068 non-null float64
price_usd_per_m2         

In [112]:
data.describe()

,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,month,year,day
count,2.170950e+05,196422.000000,196422.000000,2.762460e+05,2.762460e+05,2.762460e+05,2.589930e+05,2.500680e+05,1.955600e+05,2.471140e+05,36446.000000,167932.000000,3.550700e+04,276246.000000,276246.000000,276246.000000
mean,3.495914e+06,-34.556873,-58.528887,3.284376e+05,4.170989e+06,2.489405e+05,7.979799e+03,8.169750e+03,1.990728e+03,4.309546e+03,14.535669,2.933175,3.724319e+03,6.419155,2016.227898,17.471515
std,4.349440e+05,1.365496,1.036649,6.557548e+05,5.579327e+06,3.326170e+05,3.929949e+06,3.999465e+06,4.637115e+03,1.564181e+04,105.672435,1.449115,7.538346e+04,2.820344,0.903801,8.623244
min,3.427208e+06,-51.626650,-122.419415,5.000000e+03,4.757708e+04,2.754180e+03,0.000000e+00,0.000000e+00,8.000000e-02,5.000000e-05,1.000000,0.000000,1.000000e+00,1.000000,2012.000000,1.000000
25%,3.429595e+06,-34.629130,-58.579659,1.050000e+05,1.660724e+06,9.900000e+04,4.800000e+01,4.600000e+01,1.122800e+03,1.533333e+03,2.000000,2.000000,1.000000e+03,4.000000,2016.000000,10.000000
50%,3.430786e+06,-34.591772,-58.482191,1.700000e+05,2.677548e+06,1.600000e+05,8.300000e+01,7.600000e+01,1.784906e+03,2.094118e+03,3.000000,3.000000,1.900000e+03,7.000000,2017.000000,19.000000
75%,3.435719e+06,-34.515628,-58.418655,3.200000e+05,4.758613e+06,2.830000e+05,2.000000e+02,1.600000e+02,2.442308e+03,2.884615e+03,6.000000,4.000000,3.883500e+03,8.000000,2017.000000,24.000000
max,6.693230e+06,52.619706,35.202660,1.234568e+08,7.460816e+08,4.654544e+07,2.000000e+09,2.000000e+09,1.271027e+06,2.600000e+06,7071.000000,40.000000,1.000150e+07,12.000000,2017.000000,31.000000


In [113]:
#Filtrar datos anomalos

In [114]:
# Nan para los datos faltantes es mejor que dropear todo el dato 
# Los nros estan masomenos pensados MASOMENOS JEJE
data.loc[(data['surface_total_in_m2']>5000) | (data['surface_total_in_m2'] < 30),'surface_total_in_m2'] = np.nan
data.loc[(data['surface_covered_in_m2']>5000),'surface_covered_in_m2'] = np.nan
data.loc[data['floor']>30,'floor'] = np.nan
data.loc[data['rooms']==0,'rooms'] = np.nan
data.loc[data['rooms']>10,'rooms'] = np.nan
data.loc[(data['lat']>-33) | (data['lat']<-36),['lat','lon','lat-lon']] = np.nan
data.loc[(data['lon']>-57) | (data['lon']<-60),['lat','lon','lat-lon']] = np.nan
data.loc[(data['expenses']>14000) | (data['expenses']<1000),'expenses'] = np.nan

In [115]:
data.describe()

,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,month,year,day
count,2.170950e+05,195758.000000,195758.000000,2.762460e+05,2.762460e+05,2.762460e+05,240175.000000,249962.000000,1.955600e+05,2.471140e+05,35669.000000,167762.000000,26137.000000,276246.000000,276246.000000,276246.000000
mean,3.495914e+06,-34.579944,-58.505984,3.284376e+05,4.170989e+06,2.489405e+05,200.638264,122.239956,1.990728e+03,4.309546e+03,4.443635,2.923201,3446.388453,6.419155,2016.227898,17.471515
std,4.349440e+05,0.111582,0.151836,6.557548e+05,5.579327e+06,3.326170e+05,312.046646,138.700313,4.637115e+03,1.564181e+04,3.846696,1.404890,2586.096705,2.820344,0.903801,8.623244
min,3.427208e+06,-35.765171,-59.664010,5.000000e+03,4.757708e+04,2.754180e+03,30.000000,0.000000,8.000000e-02,5.000000e-05,1.000000,1.000000,1000.000000,1.000000,2012.000000,1.000000
25%,3.429595e+06,-34.628813,-58.579659,1.050000e+05,1.660724e+06,9.900000e+04,53.000000,46.000000,1.122800e+03,1.533333e+03,2.000000,2.000000,1550.000000,4.000000,2016.000000,10.000000
50%,3.430786e+06,-34.591734,-58.481808,1.700000e+05,2.677548e+06,1.600000e+05,92.000000,76.000000,1.784906e+03,2.094118e+03,3.000000,3.000000,2500.000000,7.000000,2017.000000,19.000000
75%,3.435719e+06,-34.516081,-58.418533,3.200000e+05,4.758613e+06,2.830000e+05,212.000000,160.000000,2.442308e+03,2.884615e+03,6.000000,4.000000,4500.000000,8.000000,2017.000000,24.000000
max,6.693230e+06,-33.677756,-57.848496,1.234568e+08,7.460816e+08,4.654544e+07,5000.000000,5000.000000,1.271027e+06,2.600000e+06,30.000000,10.000000,14000.000000,12.000000,2017.000000,31.000000


In [116]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276246 entries, 0 to 317037
Data columns (total 26 columns):
id                            276246 non-null object
created_on                    276246 non-null datetime64[ns]
property_type                 276246 non-null object
place_name                    270199 non-null object
place_with_parent_names       276246 non-null object
state_name                    276246 non-null object
geonames_id                   217095 non-null float64
lat-lon                       195758 non-null object
lat                           195758 non-null float64
lon                           195758 non-null float64
price                         276246 non-null float64
currency                      276246 non-null object
price_aprox_local_currency    276246 non-null float64
price_aprox_usd               276246 non-null float64
surface_total_in_m2           240175 non-null float64
surface_covered_in_m2         249962 non-null float64
price_usd_per_m2         

In [117]:
# Dropeo las columnas que no se usan porque el test no las tiene o no son necesarias
data.drop(['price','price_usd_per_m2','price_per_m2','currency',\
          'id','created_on','geonames_id','lat-lon','price_aprox_local_currency',\
         'title','place_with_parent_names'], axis=1, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276246 entries, 0 to 317037
Data columns (total 15 columns):
property_type            276246 non-null object
place_name               270199 non-null object
state_name               276246 non-null object
lat                      195758 non-null float64
lon                      195758 non-null float64
price_aprox_usd          276246 non-null float64
surface_total_in_m2      240175 non-null float64
surface_covered_in_m2    249962 non-null float64
floor                    35669 non-null float64
rooms                    167762 non-null float64
expenses                 26137 non-null float64
description              276232 non-null object
month                    276246 non-null int64
year                     276246 non-null int64
day                      276246 non-null int64
dtypes: float64(8), int64(3), object(4)
memory usage: 43.7+ MB


In [118]:

#test.drop(['id','created_on','lat-lon','description','country_name'], axis=1, inplace=True)
data.dropna(thresh=8,inplace=True)

In [119]:
#Esto es importante, convierto los datos categoricos en numericos cada categoria una columna con 1 si true o 0 si false
##https://stackoverflow.com/questions/34265102/xgboost-categorical-variables-dummification-vs-encoding
##https://machinelearningmastery.com/data-preparation-gradient-boosting-xgboost-python/
data = data.join(pd.get_dummies(data.property_type))
data.drop('property_type',axis=1, inplace=True)
data = data.join(data.description.str.get_dummies())
data.drop('description',axis=1, inplace=True)
data = data.join(pd.get_dummies(data.place_name))
data.drop('place_name',axis=1, inplace=True)
data = data.join(pd.get_dummies(data.state_name))
data.drop('state_name',axis=1, inplace=True)

In [120]:
#####XGBOOST
#https://github.com/dmlc/xgboost
#https://xgboost.readthedocs.io/en/latest/build.html
#https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
#https://www.slideshare.net/ShangxuanZhang/kaggle-winning-solution-xgboost-algorithm-let-us-learn-from-its-author


In [121]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from xgboost import XGBRegressor
import scipy.stats as st

from sklearn.metrics import mean_squared_error

In [122]:
#divide el set de datos en parte de entrenamiento y parte de test
# no se puede usar el test bajado de kaggle porque no tiene el precio
X = data.drop(["price_aprox_usd"], axis=1)
Y = data["price_aprox_usd"]

In [32]:
from sklearn.model_selection import GridSearchCV

## Random search de hiperparametros y entrenamiento del algoritmo, 
## cuando termina tira dos scores (no se cual es mejor) y los mejores hiperparametros que dieron esos scores
## ES LA PARTE QUE TARDA
#http://danielhnyk.cz/how-to-use-xgboost-in-python/
one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

params = {  
    "n_estimators": [60],#nros random desde a hasta b
    "max_depth": [4,6,8,10],
    "learning_rate": [0.1,0.05,0.2], # nros random de una distribucion uniforme entre a y a+b
    "colsample_bytree": [.4,.5,.75,1],
    "subsample": [.75,1], 
    "gamma": [0,0.1,1],
    'reg_alpha': [20,25,30,35,40],
    "min_child_weight": [5,6,7,8]
}

xgb = XGBRegressor(nthread=2) 

grid = GridSearchCV(xgb, params, n_jobs=1,verbose=10,cv=5) # n_iters es la cant de veces que busca, 10 es lo default
grid.fit(X, Y, eval_metric="rmse")

Fitting 5 folds for each of 17280 candidates, totalling 86400 fits
[CV] min_child_weight=5, n_estimators=25, gamma=0, learning_rate=0.1, max_depth=4, colsample_bytree=0.4, reg_alpha=20, subsample=0.75 
[CV]  min_child_weight=5, n_estimators=25, gamma=0, learning_rate=0.1, max_depth=4, colsample_bytree=0.4, reg_alpha=20, subsample=0.75, score=0.3513191398463097, total=  10.8s
[CV] min_child_weight=5, n_estimators=25, gamma=0, learning_rate=0.1, max_depth=4, colsample_bytree=0.4, reg_alpha=20, subsample=0.75 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.6s remaining:    0.0s


[CV]  min_child_weight=5, n_estimators=25, gamma=0, learning_rate=0.1, max_depth=4, colsample_bytree=0.4, reg_alpha=20, subsample=0.75, score=0.5560175735960322, total=  10.8s
[CV] min_child_weight=5, n_estimators=25, gamma=0, learning_rate=0.1, max_depth=4, colsample_bytree=0.4, reg_alpha=20, subsample=0.75 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   23.4s remaining:    0.0s


[CV]  min_child_weight=5, n_estimators=25, gamma=0, learning_rate=0.1, max_depth=4, colsample_bytree=0.4, reg_alpha=20, subsample=0.75, score=0.364911469975851, total=  10.7s
[CV] min_child_weight=5, n_estimators=25, gamma=0, learning_rate=0.1, max_depth=4, colsample_bytree=0.4, reg_alpha=20, subsample=0.75 


...

[CV]  min_child_weight=5, n_estimators=25, gamma=0, learning_rate=0.1, max_depth=4, colsample_bytree=0.4, reg_alpha=20, subsample=1, score=0.36986353179242126, total=  10.0s
[CV] min_child_weight=5, n_estimators=25, gamma=0, learning_rate=0.1, max_depth=4, colsample_bytree=0.4, reg_alpha=20, subsample=1 


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.5min remaining:    0.0s


[CV]  min_child_weight=5, n_estimators=25, gamma=0, learning_rate=0.1, max_depth=4, colsample_bytree=0.4, reg_alpha=20, subsample=1, score=0.5553337038825829, total=   9.5s
[CV] min_child_weight=5, n_estimators=25, gamma=0, learning_rate=0.1, max_depth=4, colsample_bytree=0.4, reg_alpha=20, subsample=1 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.7min remaining:    0.0s


KeyboardInterrupt: 

In [33]:
## Random search de hiperparametros y entrenamiento del algoritmo, 
## cuando termina tira dos scores (no se cual es mejor) y los mejores hiperparametros que dieron esos scores
## ES LA PARTE QUE TARDA
#http://danielhnyk.cz/how-to-use-xgboost-in-python/
one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

params = {  
    "n_estimators": [200],#nros random desde a hasta b
    "max_depth": st.randint(20,25),
    "learning_rate": st.uniform(0.05,0.05), # nros random de una distribucion uniforme entre a y a+b
    "colsample_bytree": [1],
    "subsample": [1], 
    "gamma": st.uniform(5,5),
    'reg_alpha': [28,30,31],
    "min_child_weight": [8]
}

xgb = XGBRegressor(nthread=2) 

grid = RandomizedSearchCV(xgb, params, n_jobs=1,n_iter=20,verbose=10,cv=4) # n_iters es la cant de veces que busca, 10 es lo default
grid.fit(X, Y, eval_metric="rmse")

Fitting 4 folds for each of 20 candidates, totalling 80 fits
[CV] colsample_bytree=1, max_depth=23, reg_alpha=30, subsample=1, learning_rate=0.060473203185, n_estimators=200, gamma=8.3811175153, min_child_weight=8 
[CV]  colsample_bytree=1, max_depth=23, reg_alpha=30, subsample=1, learning_rate=0.060473203185, n_estimators=200, gamma=8.3811175153, min_child_weight=8, score=0.4644341502435654, total=14.8min
[CV] colsample_bytree=1, max_depth=23, reg_alpha=30, subsample=1, learning_rate=0.060473203185, n_estimators=200, gamma=8.3811175153, min_child_weight=8 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 15.4min remaining:    0.0s


[CV]  colsample_bytree=1, max_depth=23, reg_alpha=30, subsample=1, learning_rate=0.060473203185, n_estimators=200, gamma=8.3811175153, min_child_weight=8, score=0.6917327732683709, total=15.5min
[CV] colsample_bytree=1, max_depth=23, reg_alpha=30, subsample=1, learning_rate=0.060473203185, n_estimators=200, gamma=8.3811175153, min_child_weight=8 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 31.5min remaining:    0.0s


KeyboardInterrupt: 

In [32]:
print(grid.best_score_)
#print(grid.best_estimator_)
print(grid.best_params_)
#print(grid.cv_results_)

0.595078984469
{'colsample_bytree': 1, 'max_depth': 15, 'reg_alpha': 30, 'subsample': 1, 'learning_rate': 0.10758032631237824, 'n_estimators': 60, 'gamma': 2.755708241064212, 'min_child_weight': 8}


In [55]:
# Parte para probar los parametros sin random search
# no tarda mucho porque tira el score de la combinacion de parametros que le pones nada mas
mse = []
for i in range(0,5):
    print(i)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.30)
    xgb = XGBRegressor(colsample_bytree=1,gamma=0.15,learning_rate=0.15,max_depth=12,min_child_weight=8,n_estimators=65,\
                  reg_alpha=30,subsample=1,nthread=2) 

    xgb.fit(X_train, Y_train,eval_metric="rmse")
    e = mean_squared_error(Y_test, xgb.predict(X_test))
    print(e)
    mse.append(e)
print(np.mean(mse))
print(mse)

0
53823469590.2
1
54448043600.8
2
23489812167.1
3
46632252181.7
4
48223247404.2
45323364988.8
[53823469590.241493, 54448043600.774498, 23489812167.115326, 46632252181.712158, 48223247404.192787]


In [30]:
len(X)*0.06

16013.22

In [37]:
# Print the rmse score 24415824749.1
#                      51317855617.9
#27872839091.6
#71731022891
#22648919291.2
print(mean_squared_error(Y_test, xgb.predict(X_test)))

29827010463.3


In [123]:
#Prediccion

In [124]:
test = pd.read_csv('data/properati_dataset_testing_noprice.csv')

In [125]:
#level_0, ph, Buenos Aires Interior, index, casa, departamento
test.loc[test.state_name.str.contains('Interior'),'place_name'] = 'Palermo'
test.loc[test.state_name.str.contains('Interior'),'state_name'] = 'Capital Federal'
test.loc[test.property_type.str.contains('departamento'),'property_type'] = 'apartment'
test.loc[test.property_type.str.contains('casa'),'property_type'] = 'house'
test.loc[test.property_type.str.contains('ph'),'property_type'] = 'PH'


In [126]:
def convert(s):
    try:
        return float(s)
    except:
        return np.nan

In [127]:
test['precio_from_desc'] = test.description.apply(precio_from_desc)
test.loc[test.description.notnull(),'description'] = test.loc[test.description.notnull(),'description'].apply(descripcion_token)
test.loc[:,'place_name'] = test.place_with_parent_names.apply(place_name_from_parent)
test.loc[:,'place_name'] = test.place_name.apply(rescatar_barrios_no_oficiales)
test.loc[test.surface_total_in_m2.isnull(),'surface_total_in_m2'] = test.surface_covered_in_m2
test.loc[test.rooms.isnull(),'rooms'] = test.description.apply(recuperar_cant_amb)
test['created_on'] = pd.to_datetime(test['created_on']) ## conversion de tipo
test['month'] = test.created_on.apply(lambda x: x.month)
test['year'] = test.created_on.apply(lambda x: x.year)
test['day'] = test.created_on.apply(lambda x: x.day)
test.drop(['id','created_on','operation','lat-lon','place_with_parent_names','country_name'], axis=1, inplace=True)
test.expenses = test.expenses.apply(convert)

In [128]:
test.describe()

,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,precio_from_desc,month,year,day
count,10487.000000,10487.000000,1.339200e+04,1.300500e+04,1368.000000,7500.000000,2.523000e+03,1.614000e+03,14166.0,14166.0,14166.000000
mean,-34.629923,-58.465820,3.934301e+02,3.594716e+02,10.679094,2.804667,8.522019e+03,1.833799e+05,8.0,2017.0,14.021530
std,0.101952,0.164709,2.805803e+04,2.846901e+04,93.466971,1.425710,1.916011e+05,2.147656e+05,0.0,0.0,8.030705
min,-38.416097,-63.616672,0.000000e+00,1.000000e+00,1.000000,1.000000,0.000000e+00,2.015000e+04,8.0,2017.0,1.000000
25%,-34.643998,-58.499255,4.600000e+01,4.200000e+01,2.000000,2.000000,1.000000e+03,7.600000e+04,8.0,2017.0,9.000000
50%,-34.612651,-58.443276,7.200000e+01,6.400000e+01,4.000000,3.000000,2.000000e+03,1.250000e+05,8.0,2017.0,14.000000
75%,-34.589405,-58.401978,1.450000e+02,1.120000e+02,7.000000,4.000000,3.600000e+03,2.100000e+05,8.0,2017.0,20.000000
max,-34.536105,-57.900605,3.243242e+06,3.243242e+06,2509.000000,21.000000,7.600072e+06,2.700000e+06,8.0,2017.0,29.000000


In [129]:
# Nan para los datos faltantes es mejor que dropear todo el dato 
# Los nros estan masomenos pensados MASOMENOS JEJE
test.loc[(test['surface_total_in_m2']>5000) | (test['surface_total_in_m2'] < 30),'surface_total_in_m2'] = np.nan
test.loc[(test['surface_covered_in_m2']>5000),'surface_covered_in_m2'] = np.nan
test.loc[test['floor']>30,'floor'] = np.nan
test.loc[test['rooms']==0,'rooms'] = np.nan
test.loc[test['rooms']>10,'rooms'] = np.nan
test.loc[(test['lat']>-33) | (test['lat']<-36),['lat','lon']] = np.nan
test.loc[(test['lon']>-57) | (test['lon']<-60),['lat','lon']] = np.nan
test.loc[(test['expenses']>14000) | (test['expenses']<100),'expenses'] = np.nan

In [130]:
test.describe()

,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,precio_from_desc,month,year,day
count,10483.000000,10483.000000,12817.000000,12998.000000,1354.000000,7490.000000,2279.000000,1.614000e+03,14166.0,14166.0,14166.000000
mean,-34.628479,-58.463854,134.013654,95.626943,5.118907,2.788919,2882.024133,1.833799e+05,8.0,2017.0,14.021530
std,0.070184,0.130426,187.645059,126.166300,4.215539,1.355359,2476.270406,2.147656e+05,0.0,0.0,8.030705
min,-35.123100,-58.975341,30.000000,1.000000,1.000000,1.000000,120.000000,2.015000e+04,8.0,2017.0,1.000000
25%,-34.643941,-58.499089,48.000000,42.000000,2.000000,2.000000,1200.000000,7.600000e+04,8.0,2017.0,9.000000
50%,-34.612629,-58.443256,75.000000,64.000000,4.000000,3.000000,2000.000000,1.250000e+05,8.0,2017.0,14.000000
75%,-34.589392,-58.401970,150.000000,111.000000,7.000000,4.000000,3621.000000,2.100000e+05,8.0,2017.0,20.000000
max,-34.536105,-57.900605,4810.000000,4733.000000,29.000000,10.000000,14000.000000,2.700000e+06,8.0,2017.0,29.000000


In [131]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 15 columns):
property_type            14166 non-null object
place_name               13901 non-null object
state_name               14166 non-null object
lat                      10483 non-null float64
lon                      10483 non-null float64
surface_total_in_m2      12817 non-null float64
surface_covered_in_m2    12998 non-null float64
floor                    1354 non-null float64
rooms                    7490 non-null float64
expenses                 2279 non-null float64
description              14166 non-null object
precio_from_desc         1614 non-null float64
month                    14166 non-null int64
year                     14166 non-null int64
day                      14166 non-null int64
dtypes: float64(8), int64(3), object(4)
memory usage: 1.6+ MB


In [132]:
test = test.join(pd.get_dummies(test.property_type))
test.drop('property_type',axis=1, inplace=True)
test = test.join(test.description.str.get_dummies())
test.drop('description',axis=1, inplace=True)
test = test.join(pd.get_dummies(test.place_name))
test.drop('place_name',axis=1, inplace=True)
test = test.join(pd.get_dummies(test.state_name))
test.drop('state_name',axis=1, inplace=True)

In [133]:
tot = pd.concat([X,test])
tot.reset_index(inplace=True,drop=True)

In [134]:
test_2 = tot[(len(tot)-len(test)):]
test_2.reset_index(inplace=True,drop=True)

In [135]:
test_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Columns: 142 entries, Abasto to year
dtypes: float64(20), int64(38), uint8(84)
memory usage: 7.4 MB


In [136]:
test_2 = test_2[X.columns]

In [137]:
test_2.surface_covered_in_m2.describe()

count    12998.000000
mean        95.626943
std        126.166300
min          1.000000
25%         42.000000
50%         64.000000
75%        111.000000
max       4733.000000
Name: surface_covered_in_m2, dtype: float64

In [138]:
xgb = XGBRegressor(colsample_bytree=1,gamma=9.3358906320029078,learning_rate=0.054757194433536716,max_depth=22,min_child_weight=20,n_estimators=200,\
                  reg_alpha=30,subsample=0.96242118485640649,nthread=2) 

xgb.fit(X, Y,eval_metric="rmse")

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=9.335890632002908, learning_rate=0.054757194433536716,
       max_delta_step=0, max_depth=22, min_child_weight=20, missing=None,
       n_estimators=200, nthread=2, objective='reg:linear', reg_alpha=30,
       reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=0.9624211848564065)

In [139]:
pred = xgb.predict(test_2)

In [142]:
test_f = pd.read_csv('data/properati_dataset_testing_noprice.csv')
test_f['precio_from_desc'] = test_f.description.apply(precio_from_desc)


In [147]:
test_f.iloc[42].description

'                MONROE ESQUINA MELIAN - MONOAMBIENTE AMPLIO CON BALCÓN AL FRENTE - ORIENTACIÓN ESTE - SUPER LUMINOSO - EDIFICIO APTO PROFESIONAL - COCHERAS Y BAULERAS OPTATIVAS - COCINA A GAS E INSTALACIÓN PARA LAVARROPAS - AGUA CALIENTE CENTRAL POR TERMOTANQUES DE ALTA RECUPERACIÓN - SUM CON COCINA Y BAÑO, PISCINA, PARRILLAS, AMPLIAS TERRAZAS Y SOLARIUM - EXCELENTE UBICACIÓN - ESCRITURA Y POSESIÓN INMEDIATA - BAJAS EXPENSAS - COCHERAS 1ER SUBSUELO USD 26.000COCHERAS 2DO SUBSUELO USD 23.000BAULERAS USD 3.000 de la paz 1970consúltenos:             Excelencia en Servicios Inmobiliarios, desde 1972.'

In [148]:
test_e = pd.read_csv('data/properati_dataset_testing_noprice.csv')
test_e['precio_from_desc'] = test_e.description.apply(precio_from_desc)
test_e['price_usd'] = pred
test_e.loc[test_e.precio_from_desc.notnull() & (abs(100-test_e.precio_from_desc*100/test_e.price_usd) < 40),'price_usd'] = test_e['precio_from_desc']
test_e = test_e.loc[:,['id','price_usd']]
test_e.to_csv('Predicciones/XgBoost_10_11_b.csv', index = False)

In [140]:
test_e['price_usd'] = pred

In [64]:
test_e.drop('description',axis=1,inplace=True)

In [65]:
test_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 2 columns):
id           14166 non-null int64
price_usd    14166 non-null float32
dtypes: float32(1), int64(1)
memory usage: 166.1 KB


In [2]:
ff = pd.read_csv('Predicciones/XgBoost_2.csv')
pred = ff['price_usd']

AttributeError: 'Series' object has no attribute 'info'

In [7]:
test_e = pd.read_csv('data/properati_dataset_testing_noprice.csv')
test_e['precio_from_desc'] = test_e.description.apply(precio_from_desc)
test_e['price_usd'] = pred
test_e.loc[test_e.precio_from_desc.notnull() & (abs(100-test_e.precio_from_desc*100/test_e.price_usd) < 30),'price_usd'] = test_e['precio_from_desc']
test_e = test_e.loc[:,['id','price_usd']]
test_e.to_csv('Predicciones/XgBoost_2_b.csv', index = False)

In [8]:
test_e.to_csv('Predicciones/XgBoost_2_b.csv', index = False)

In [156]:
test_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 2 columns):
id           14166 non-null int64
price_usd    14166 non-null float32
dtypes: float32(1), int64(1)
memory usage: 166.1 KB


In [197]:
test_e = pd.read_csv('data/properati_dataset_testing_noprice.csv')

In [175]:
test_e.iloc[5].description

'                AMBIENTE DIVISIBLE CON PISOS DE PORCELANATO NATURAL CON ZÓCALOS DE MADERA COLOR WENGUE- AMPLIO BAÑO COMPLETO REVESTIDO CON CERÁMICA SAN LORENZO COLOR ARENA Y GUARDAS DE VENECITAS- ARTEFACTOS SANITARIOS Y GRIFERÍAS SERÁN DE MARCAS DE PRIMERA LÍNEA (ROCA, GENEBRE, FERRUM O SIMILAR)- PLACARD COMPLETO-COCINA CON MESADA DE GRANITO, MUEBLES BAJO Y SOBRE MESADA COMPLETOS-  COCINA CON HORNO Y ANAFE DE 4 HORNALLAS A GAS- PRE-INSTALACIÓN PARA EQUIPOS DE AIRE ACONDICIONADOAGUA CALIENTE CENTRAL-BALCÓN CON CONEXIÓN PARA LAVARROPAS- AMENITIES: SOLÁRIUM CON PARRILLA.ENTREGA SEPTIEMBRE 2018PRECIO DE CONTADO- CONSULTAR POR FINANCIACIÓN ADELANTO DE 35% Y CUOTAS CON AJUSTE CAC.COCHERAS OPCIONALES DESDE u$s20.000.-BAULERAS OPCIONALES u$S2.200.-CONSÚLTENOS- SUCURSAL PALERMO-             Excelencia en Servicios Inmobiliarios, desde 1972.'

In [206]:
test_e['precio_from_desc'] = test_e.description.apply(precio_from_desc)

In [207]:
test_e.sort_values(by='precio_from_desc',ascending=True).head(20)

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description,precio_from_desc
4638,2289157,2017-08-11,departamento,venta,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,"-34.6427936,-58.5117661",-34.642794,-58.511766,55.0,50.0,NaN,3.0,NaN,Se ofrecen a la venta dos unidades de tres amb...,125.0
1572,2273266,2017-08-05,departamento,venta,Flores,|Argentina|Capital Federal|Flores|,Argentina,Capital Federal,"-34.618926,-58.461959",-34.618926,-58.461959,176.0,157.0,NaN,5.0,NaN,Corredor Responsable: Rebeca Stern - C.U.C.I.C...,147.0
4653,2289184,2017-08-11,departamento,venta,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,"-34.6505806,-58.5182644",-34.650581,-58.518264,100.0,84.0,NaN,3.0,NaN,PRIMER PISO -3amb. Al frente 100m2 totales U$S...,250.0
829,2268333,2017-08-02,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5854083675,-58.4269523302",-34.585408,-58.426952,91.0,70.0,NaN,NaN,5500,VENTA DEPARTAMENTO DE 3 AMBIENTES EN PALERO CO...,265.0
821,2268312,2017-08-02,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5901346135,-58.43209812",-34.590135,-58.432098,104.0,73.0,NaN,NaN,NaN,VENTA DEPARTAMENTO 3 AMBIENTES EN PALERMO SOHO...,315.0
2531,2279130,2017-08-09,departamento,venta,San Cristobal,|Argentina|Capital Federal|San Cristobal|,Argentina,Capital Federal,"-34.6237771,-58.4022249",-34.623777,-58.402225,52.0,52.0,13.0,1.0,NaN,Zona:Buena-Asfalto-Osn:Cloaca-Lavadero-Medidas...,350.0
2532,2279131,2017-08-09,departamento,venta,San Cristobal,|Argentina|Capital Federal|San Cristobal|,Argentina,Capital Federal,"-34.6237771,-58.4022249",-34.623777,-58.402225,52.0,52.0,13.0,1.0,NaN,Zona:Buena-Asfalto-Osn:Cloaca-Lavadero-Medidas...,350.0
5026,2290737,2017-08-11,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5900996057,-58.4302024449",-34.590100,-58.430202,160.0,130.0,NaN,NaN,4000,VENTA DUPLEX 4 AMBIENTES EN PALERMO SOHO / CON...,410.0
12185,2183588,2017-08-01,casa,venta,Lomas de Zamora,|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zam...,Argentina,Bs.As. G.B.A. Zona Sur,"-34.7722138,-58.4309831",-34.772214,-58.430983,385.0,190.0,NaN,NaN,NaN,Venta de Casa 5 AMBIENTES CON COCHERA Y FOND...,418.0
7513,2304678,2017-08-17,casa,venta,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,NaN,NaN,NaN,NaN,340.0,NaN,NaN,5000,USD 480 mil. o alquilo amueblado por dos años ...,480.0


In [204]:
test_e.iloc[89].description

'Excelente departamento ubicado en lo mejor de Av. Del Libertador, Cuenta con Living comedor, con vista abierta, balcón corrido al frente . Sus tres dormitorios cuenta con balcón corrido (contrafrente) con vista al rio al igual que la cocina- comedor.Posee amplio lavadero , un baño completo y un toa Excelente luz natural, y vista.Cuenta con cochera fija, con capacidad para dos autos , mas baulera. Puerta Blindada en la entrada principal.Impecable y en lo mejor de Belgrano!!!- Se deja constancia que los m2 indicados son aproximados, al igual que las medidas parciales de los ambientes, y el valor consignado de expensas mensuales está sujeto a verificación y/o ajustes.Fotos de carácter no contractual -Consúltenos.'